In [1]:
from transformers import pipeline
import pandas as pd
import torch
from tqdm import tqdm

2023-04-14 17:07:13.076196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 17:07:15.726571: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-14 17:07:15.726637: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-14 17:07:15.726644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
print("Reading data")
data = pd.read_csv("data/Russia_invade.csv")

Reading data


In [7]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [16]:
print("Creating dataloader")
tweets = data["renderedContent"].tolist()[:10]
candidate_labels = ['USA started the war',
                    'POTUS started the war',
                    'Joe Biden started the war',
                    'CIA started the war',
                    'USA influenced the war',
                    'POTUS influenced the war',
                    'Joe Biden influenced the war',
                    'CIA influenced the war']

Creating dataloader


In [20]:
print("Starting analysis")
tweets_blaming_america = {}
for tweet in tqdm(tweets):
    result = classifier(tweet, candidate_labels, multi_label=True)
    if any(val > 0.75 for val in result['scores']):
        tweets_blaming_america[tweet] = {l: s for l, s in zip(result['labels'], result['scores'])}

Starting analysis


100%|██████████| 10/10 [00:19<00:00,  1.97s/it]


In [21]:
print(tweets_blaming_america)

{'JOE BIDEN SAYS HOW DO WE GET TO A PLACE WHERE PUTIN DECIDES TO INVADE RUSSIA 🤣🤣🤣😭😭😂😂\nPEOPLE VOTED FOR THIS GUY 🤣🤣🤣 https://t.co/FOHozS7vYI': {'Joe Biden influenced the war': 0.958122730255127, 'Joe Biden started the war': 0.06567798554897308, 'USA influenced the war': 0.061619728803634644, 'CIA influenced the war': 0.04029559716582298, 'POTUS influenced the war': 0.019074110314249992, 'CIA started the war': 0.0029467542190104723, 'POTUS started the war': 0.0004867762327194214, 'USA started the war': 0.0004355523851700127}, 'America forced Putin to invade in order to bog down Russia. Because Americans are evil and are sitting in their White House cackling evilly, devising wicked plans.\n\n10/10 genius logic Glenn https://t.co/zHHGG5amqX': {'USA influenced the war': 0.9927944540977478, 'USA started the war': 0.9240317344665527, 'POTUS influenced the war': 0.9187301397323608, 'POTUS started the war': 0.42189261317253113, 'CIA influenced the war': 0.049384839832782745, 'CIA started the 

In [22]:
for tweet, result in tweets_blaming_america.items():
    print(tweet.replace("\n", " "))
    for label, score in result.items():
        print(f"\t{label}: {score}")

JOE BIDEN SAYS HOW DO WE GET TO A PLACE WHERE PUTIN DECIDES TO INVADE RUSSIA 🤣🤣🤣😭😭😂😂 PEOPLE VOTED FOR THIS GUY 🤣🤣🤣 https://t.co/FOHozS7vYI
	Joe Biden influenced the war: 0.958122730255127
	Joe Biden started the war: 0.06567798554897308
	USA influenced the war: 0.061619728803634644
	CIA influenced the war: 0.04029559716582298
	POTUS influenced the war: 0.019074110314249992
	CIA started the war: 0.0029467542190104723
	POTUS started the war: 0.0004867762327194214
	USA started the war: 0.0004355523851700127
America forced Putin to invade in order to bog down Russia. Because Americans are evil and are sitting in their White House cackling evilly, devising wicked plans.  10/10 genius logic Glenn https://t.co/zHHGG5amqX
	USA influenced the war: 0.9927944540977478
	USA started the war: 0.9240317344665527
	POTUS influenced the war: 0.9187301397323608
	POTUS started the war: 0.42189261317253113
	CIA influenced the war: 0.049384839832782745
	CIA started the war: 0.039775583893060684
	Joe Biden in

In [23]:
tests = ['POTUS is the one that has been antagonising Russia. They are the ones who started this war, now we will be the ones to finish it!',
         'America started the war, and now theyre the ones that are calling Russia evil! Hypocrites!!',
         'CIA meddling in Ukraine since WWII, cia paramilitary, Zelensky being an installed actor.']
for tweet in tests:
    candidate_labels = ['USA started the war', 'Joe Biden is a bad President',
                        'USA are evil', 'POTUS is bad', 'CIA influenced the war']
    result = classifier(tweet, candidate_labels, multi_label=True)
    print(tweet)
    for label, score in zip(result['labels'], result['scores']):
        print(f"\t{label}: {score}")

POTUS is the one that has been antagonising Russia. They are the ones who started this war, now we will be the ones to finish it!
	POTUS is bad: 0.993208646774292
	USA started the war: 0.8688077926635742
	USA are evil: 0.7750784158706665
	CIA influenced the war: 0.005277483724057674
	Joe Biden is a bad President: 9.682839299784973e-05
America started the war, and now theyre the ones that are calling Russia evil! Hypocrites!!
	USA started the war: 0.9981517791748047
	USA are evil: 0.7820971608161926
	POTUS is bad: 0.10557670891284943
	CIA influenced the war: 0.03276588395237923
	Joe Biden is a bad President: 0.006584193557500839
CIA meddling in Ukraine since WWII, cia paramilitary, Zelensky being an installed actor.
	CIA influenced the war: 0.9955834150314331
	USA are evil: 0.23233827948570251
	POTUS is bad: 0.01661643013358116
	Joe Biden is a bad President: 0.0012317727087065578
	USA started the war: 0.0010602643014863133
